## **利用ogr开源库进行矢量文件的读写**
**1. 投影转换**  
**2. 计算面要素的面积信息，添加面积字段/删除面积字段**  
**3. 获取面要素中心点，新建中心点图层**  
**4. 根据地理信息新建面要素**  


In [4]:
from osgeo import ogr
from osgeo import osr


In [5]:
path_kunming_districts = 'data/Section-2/kunming_districts.shp'


### 1. Re-project the vector to utm projection.
EPSG是European Petroleum Survey Group (欧洲石油调查小组)的缩写,该组织发布了一个坐标参照系统的数据集,数据集对全球收录到的坐标参照系统进行了编码。  
wgs84: 4326; 
wgs84+utm: 北半球：326+投影带（如32622）；南半球：327+投影带（32722）


In [107]:
#### Reprojection
path_kunming_districts_utm = 'data/Section-4/kunming_districts_utm.shp'
utm_epsg = 32648
in_ds = ogr.Open(path_kunming_districts)
in_layer = in_ds.GetLayer(0)
### 创建输出矢量
driver = ogr.GetDriverByName('ESRI Shapefile')
out_ds = driver.CreateDataSource(path_kunming_districts_utm)
out_layer = out_ds.CreateLayer('kunming_districts_utm', geom_type = ogr.wkbPolygon)
### 在图层上创建属性字段
for i in range(in_layer.GetFeature(0).GetFieldCount()):
  field_defn = in_layer.GetFeature(0).GetFieldDefnRef(i)
  out_layer.CreateField(field_defn)

## 空间参考
spatialRef = in_layer.GetSpatialRef()
utm_spatialRef = osr.SpatialReference()
utm_spatialRef.ImportFromEPSG(utm_epsg)
# 投影转换定义
coordinate_transform = osr.CoordinateTransformation(spatialRef, utm_spatialRef)
## 获得要素定义
FeaDefn = in_layer.GetLayerDefn()
### 遍历每个要素
for i in range(in_layer.GetFeatureCount()):
  in_fea = in_layer.GetFeature(i)
  in_geo = in_fea.geometry()            ## 获得原矢量要素的几何
  in_geo.Transform(coordinate_transform)
  out_geo = in_geo 
  out_fea = ogr.Feature(FeaDefn)        ## 定义输出要素
  out_fea.SetGeometry(in_geo)           ## 在输出要素中添加几何
  ### 遍历每个属性字段
  for j in range(FeaDefn.GetFieldCount()):
    field_value = in_fea.GetField(j)
    out_fea.SetField(j, field_value)    ## 在输出要素中添加输出属性
  out_layer.CreateFeature(out_fea)
ds = None
out_ds = None

# 保存投影文件
prj_path = path_kunming_districts_utm.replace(".shp", ".prj")
fn = open(prj_path, 'w')
print(utm_spatialRef.ExportToWkt())
fn.write(utm_spatialRef.ExportToWkt())
fn.close()



PROJCS["WGS 84 / UTM zone 48N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",105],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32648"]]


#### 2. 添加/删除面积字段，并计算面积

In [108]:
ds = ogr.Open(path_kunming_districts_utm, 1)  ## 0是只读，1是可写
layer = ds.GetLayer(0)
field_defn = ogr.FieldDefn('Area', ogr.OFTReal)   ## OFTReal：浮点型；OFTInteger: 整型
layer.CreateField(field_defn)       ## 创建新字段
# id_area = fea.GetFieldIndex('Area')
# layer.DeleteField(id_area)   ## 删除字段，参数只能是索引
ds = None    ### 关闭数据源（重要）


In [109]:
ds = ogr.Open(path_kunming_districts_utm, 1)  ## 0是只读，1是可写
layer = ds.GetLayer(0)
### 遍历每一个要素
for i in range(layer.GetFeatureCount()):
  fea = layer.GetFeature(i)
  fea_area = fea.geometry().Area()
  fea_area = fea_area/1000000       ### 单位：千米
  print('Area is: ', fea_area)
  fea.SetField('Area', fea_area)   ## 对面积字段Field赋值
  layer.SetFeature(fea)  ## 更新特征
ds = None    ### 关闭数据源（重要）


Area is:  377.9959786954056
Area is:  864.2952967369265
Area is:  638.3336172947612
Area is:  897.3967316211479
Area is:  1866.1175875336044
Area is:  499.36396388767395
Area is:  1338.5644973111575
Area is:  994.6751193008348
Area is:  1914.0921754168446
Area is:  1681.3428560105956
Area is:  828.4756858595072
Area is:  4227.345074248606
Area is:  3598.3429000372385
Area is:  1301.0062596518349


#### 3. 获取面要素中心点，新建中心点图层

In [116]:
path_kunming_districts_centroid = 'data/Section-4/kunming_districts_centroid.shp'
in_ds = ogr.Open(path_kunming_districts, 0)  ## 0是只读，1是可写
in_layer = in_ds.GetLayer(0); 
## 空间参考
in_SpatialRef = in_layer.GetSpatialRef()
### 创建新矢量文件
driver = ogr.GetDriverByName('ESRI Shapefile')
out_ds = driver.CreateDataSource(path_kunming_districts_centroid)
out_layer = out_ds.CreateLayer('kunming_districts_centroid', geom_type=ogr.wkbPoint, srs=in_SpatialRef)  # create new layer in the shapefile 
field_defn = ogr.FieldDefn('Fid', ogr.OFTInteger)   ## 定义属性字段
out_layer.CreateField(field_defn)     ## 创建新字段
out_fea_defn = out_layer.GetLayerDefn()
## 遍历要素
for i in range(in_layer.GetFeatureCount()):
  in_fea = in_layer.GetFeature(i)
  out_geo = in_fea.geometry().Centroid()  ## 获得中心点几何  
  out_fea = ogr.Feature(out_fea_defn)     ## 创建空要素
  out_fea.SetGeometry(out_geo)            ## 设置要素几何
  out_fea.SetField('Fid', i+1)            ## 设置字段值
  out_layer.CreateFeature(out_fea)        ## 在图层中创建要素。
out_ds = None                            ## 关闭数据源
ds = None



#### 4. 新建几何及多要素图层
云大呈贡校园范围（(102.837, 24.832), (102.842, 24.836), (102.849, 24.836), (102.857, 24.835), (102.864, 24.828),(102.855, 24.826), (102.845, 24.821),(102.837, 24.832)）  
wkt(OGC well-known text)和wkb(OGC well-known binary)是OGC制定的空间数据的组织规范，wkt是以文本形式描述，wkb是以二进制形式描述。   
示例:  
Point: POINT (30 10)  
LineString: LINESTRING (30 10, 10 30, 20 20, 40 40)  
Polygon: POLYGON ((30 10, 40 40, 20 40, 10 20, 35 10))  

In [115]:
path_ynu_vector = 'data/Section-4/ynu_vec.shp'
### Create polygon by define wkt
ynu_wkt = "POLYGON ((102.837 24.832,102.842 24.836,102.849 24.836,102.857 24.835,102.864 24.828,102.855 24.826,102.845 24.821,102.837 24.832))"
geo_ynu = ogr.CreateGeometryFromWkt(ynu_wkt)
### 创建输出矢量数据
driver = ogr.GetDriverByName('ESRI Shapefile')
ds_ynu = driver.CreateDataSource(path_ynu_vector)
SpatialRef = osr.SpatialReference()   ## create spatial reference object
SpatialRef.ImportFromEPSG(4326)       ## set it to EPSG:4326
layer_ynu = ds_ynu.CreateLayer('ynu_vec', srs=SpatialRef, geom_type=ogr.wkbPolygon)
fea_defn = layer_ynu.GetLayerDefn()
field_defn = ogr.FieldDefn('Fid', ogr.OFTInteger)
layer_ynu.CreateField(field_defn)   ## 创建属性字段
fea_ynu = ogr.Feature(fea_defn)
fea_ynu.SetField('Fid', 1)
fea_ynu.SetGeometry(geo_ynu)
layer_ynu.CreateFeature(fea_ynu)
ds_ynu = None



POLYGON ((102.837 24.832 0,102.842 24.836 0,102.849 24.836 0,102.857 24.835 0,102.864 24.828 0,102.855 24.826 0,102.845 24.821 0,102.837 24.832 0))
bytearray(b'\x01\x03\x00\x00\x80\x01\x00\x00\x00\x08\x00\x00\x00!\xb0rh\x91\xb5Y@\xa2E\xb6\xf3\xfd\xd48@\x00\x00\x00\x00\x00\x00\x00\x00\xd9\xce\xf7S\xe3\xb5Y@\xbct\x93\x18\x04\xd68@\x00\x00\x00\x00\x00\x00\x00\x00u\x93\x18\x04V\xb6Y@\xbct\x93\x18\x04\xd68@\x00\x00\x00\x00\x00\x00\x00\x00\x02+\x87\x16\xd9\xb6Y@\xf6(\\\x8f\xc2\xd58@\x00\x00\x00\x00\x00\x00\x00\x00\x9e\xef\xa7\xc6K\xb7Y@\x87\x16\xd9\xce\xf7\xd38@\x00\x00\x00\x00\x00\x00\x00\x00\x1f\x85\xebQ\xb8\xb6Y@\xfa~j\xbct\xd38@\x00\x00\x00\x00\x00\x00\x00\x00\xaeG\xe1z\x14\xb6Y@\x19\x04V\x0e-\xd28@\x00\x00\x00\x00\x00\x00\x00\x00!\xb0rh\x91\xb5Y@\xa2E\xb6\xf3\xfd\xd48@\x00\x00\x00\x00\x00\x00\x00\x00')
